# 🍽️ Where to DINE - 餐厅数据采集
## 方法1: Google Maps Places API

---

### 📋 功能说明
本notebook使用Google Maps Places API获取纽约市的餐厅数据

### ⚙️ 准备工作
1. 需要Google Maps API Key（Places API）
2. 在Google Cloud Console启用Places API
3. 确保API有足够的配额

### 📊 输出数据
- 文件名: `restaurants_nyc_googlemaps.csv`
- 包含: 餐厅名称、地址、坐标、评分、价格等级、菜系类型

---

## 1️⃣ 安装依赖库

In [ ]:
# 安装必要的Python库
!pip install googlemaps pandas numpy requests -q

print("✅ 依赖库安装完成！")

## 2️⃣ 导入库并配置

In [ ]:
import googlemaps
import pandas as pd
import numpy as np
import time
from datetime import datetime
import json

print("✅ 库导入成功！")

## 3️⃣ 配置API Key和搜索参数

In [ ]:
# ========================================
# 🔑 在这里输入你的Google Maps API Key
# ========================================
API_KEY = "MY_KEY"  # 替换为你的真实API Key

# 初始化Google Maps客户端
gmaps = googlemaps.Client(key=API_KEY)

# NYC边界坐标
NYC_BOUNDS = {
    'north': 40.9176,
    'south': 40.4774,
    'east': -73.7004,
    'west': -74.2591
}

# NYC中心点（曼哈顿中城）
NYC_CENTER = (40.7589, -73.9851)

# 搜索半径（米）- Google Maps API限制最大50000米
SEARCH_RADIUS = 50000  # 50公里

print("✅ API配置完成！")
print(f"📍 搜索中心: {NYC_CENTER}")
print(f"📏 搜索半径: {SEARCH_RADIUS/1000}公里")

## 4️⃣ 生成搜索网格

由于Google Maps API每次搜索返回最多60个结果，我们需要将NYC划分成多个网格区域进行搜索

In [ ]:
def create_search_grid(bounds, grid_size=5):
    """
    创建搜索网格点
    
    参数:
        bounds: 边界字典
        grid_size: 网格数量（每边）
    
    返回:
        网格中心点列表
    """
    lats = np.linspace(bounds['south'], bounds['north'], grid_size)
    lons = np.linspace(bounds['west'], bounds['east'], grid_size)
    
    grid_points = []
    for lat in lats:
        for lon in lons:
            grid_points.append((lat, lon))
    
    return grid_points

# 创建5x5网格（25个搜索点）
search_grid = create_search_grid(NYC_BOUNDS, grid_size=5)

print(f"✅ 创建了 {len(search_grid)} 个搜索网格点")
print(f"示例点: {search_grid[0]}")

## 5️⃣ 定义搜索函数

In [ ]:
def search_restaurants_nearby(location, radius=5000):
    """
    在指定位置附近搜索餐厅
    
    参数:
        location: (lat, lon) 元组
        radius: 搜索半径（米）
    
    返回:
        餐厅列表
    """
    try:
        # 使用Places API的nearby_search
        places_result = gmaps.places_nearby(
            location=location,
            radius=radius,
            type='restaurant'
        )
        
        restaurants = places_result.get('results', [])
        
        # 如果有下一页结果，继续获取
        while 'next_page_token' in places_result and len(restaurants) < 60:
            # 需要等待一下才能使用next_page_token
            time.sleep(2)
            
            places_result = gmaps.places_nearby(
                page_token=places_result['next_page_token']
            )
            
            restaurants.extend(places_result.get('results', []))
        
        return restaurants
    
    except Exception as e:
        print(f"❌ 搜索出错: {e}")
        return []

print("✅ 搜索函数已定义")

## 6️⃣ 定义数据解析函数

In [ ]:
def parse_restaurant_data(place):
    """
    解析Google Maps API返回的餐厅数据
    
    参数:
        place: API返回的place对象
    
    返回:
        字典格式的餐厅信息
    """
    geometry = place.get('geometry', {})
    location = geometry.get('location', {})
    
    return {
        'place_id': place.get('place_id', ''),
        'name': place.get('name', ''),
        'address': place.get('vicinity', ''),
        'latitude': location.get('lat', None),
        'longitude': location.get('lng', None),
        'rating': place.get('rating', None),
        'user_ratings_total': place.get('user_ratings_total', 0),
        'price_level': place.get('price_level', None),
        'types': ', '.join(place.get('types', [])),
        'business_status': place.get('business_status', ''),
        'category': 'restaurant'
    }

print("✅ 解析函数已定义")

## 7️⃣ 开始采集数据

⚠️ **注意**: 这个过程可能需要10-20分钟，因为需要遵守API速率限制

In [ ]:
print("🚀 开始采集餐厅数据...")
print(f"总共需要搜索 {len(search_grid)} 个网格点\n")

all_restaurants = []
seen_place_ids = set()  # 用于去重

for i, grid_point in enumerate(search_grid, 1):
    print(f"\r进度: {i}/{len(search_grid)} - 正在搜索 {grid_point}", end="")
    
    # 搜索餐厅
    restaurants = search_restaurants_nearby(grid_point, radius=10000)  # 10公里半径
    
    # 解析并去重
    for place in restaurants:
        place_id = place.get('place_id')
        
        if place_id and place_id not in seen_place_ids:
            seen_place_ids.add(place_id)
            restaurant_data = parse_restaurant_data(place)
            all_restaurants.append(restaurant_data)
    
    # 遵守API速率限制（每秒最多10次请求）
    time.sleep(0.5)

print(f"\n\n✅ 数据采集完成！")
print(f"📊 共采集到 {len(all_restaurants)} 个不重复的餐厅")

## 8️⃣ 转换为DataFrame并查看数据

In [ ]:
# 转换为DataFrame
df = pd.DataFrame(all_restaurants)

# 删除坐标为空的记录
df = df.dropna(subset=['latitude', 'longitude'])

# 显示基本信息
print(f"📊 数据集信息:")
print(f"  总记录数: {len(df)}")
print(f"  列数: {len(df.columns)}")
print(f"\n列名: {list(df.columns)}")
print(f"\n数据预览:")
display(df.head(10))

# 显示统计信息
print(f"\n📈 统计信息:")
print(f"  平均评分: {df['rating'].mean():.2f}")
print(f"  有评分的餐厅: {df['rating'].notna().sum()}")
print(f"  有价格信息的餐厅: {df['price_level'].notna().sum()}")

## 9️⃣ 简单数据可视化

In [ ]:
import matplotlib.pyplot as plt

# 创建图表
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# 图1: 餐厅位置分布
axes[0].scatter(df['longitude'], df['latitude'], alpha=0.5, s=10)
axes[0].set_xlabel('经度')
axes[0].set_ylabel('纬度')
axes[0].set_title(f'NYC餐厅位置分布 (n={len(df)})')
axes[0].grid(True, alpha=0.3)

# 图2: 评分分布
df['rating'].dropna().hist(bins=20, ax=axes[1], edgecolor='black')
axes[1].set_xlabel('评分')
axes[1].set_ylabel('餐厅数量')
axes[1].set_title('餐厅评分分布')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ 可视化完成！")

## 🔟 保存数据到CSV文件

In [ ]:
# 保存到CSV
output_filename = 'restaurants_nyc_googlemaps.csv'
df.to_csv(output_filename, index=False, encoding='utf-8')

print(f"✅ 数据已保存到: {output_filename}")
print(f"📁 文件大小: {pd.io.common.get_handle(output_filename, 'r').handle.seek(0, 2) / 1024:.2f} KB")

# 如果在Colab中，下载文件
try:
    from google.colab import files
    files.download(output_filename)
    print("⬇️ 文件下载已开始...")
except:
    print("💡 如需下载，请在Colab左侧文件栏找到文件右键下载")

## 🎁 额外功能：获取详细信息

如果需要更详细的餐厅信息（如营业时间、电话、网站等），可以使用Place Details API

In [ ]:
def get_place_details(place_id):
    """
    获取餐厅详细信息
    
    ⚠️ 警告: Place Details API会消耗更多配额
    """
    try:
        place_details = gmaps.place(
            place_id=place_id,
            fields=['name', 'formatted_phone_number', 'website', 
                   'opening_hours', 'price_level', 'rating', 'reviews']
        )
        return place_details.get('result', {})
    except Exception as e:
        print(f"获取详情失败: {e}")
        return {}

# 示例：获取前5个餐厅的详细信息
print("📝 示例：获取前5个餐厅的详细信息\n")

for i, place_id in enumerate(df['place_id'].head(5), 1):
    details = get_place_details(place_id)
    print(f"{i}. {details.get('name', 'N/A')}")
    print(f"   电话: {details.get('formatted_phone_number', 'N/A')}")
    print(f"   网站: {details.get('website', 'N/A')}")
    print()
    time.sleep(0.5)  # 速率限制

print("\n💡 提示: 如需获取所有餐厅详情，需要大量API配额，请谨慎使用")

## 📊 数据采集总结

In [ ]:
print("=" * 60)
print("📊 数据采集总结报告")
print("=" * 60)
print(f"\n数据来源: Google Maps Places API")
print(f"采集时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"\n数据统计:")
print(f"  ├─ 总餐厅数: {len(df)}")
print(f"  ├─ 有评分数据: {df['rating'].notna().sum()} ({df['rating'].notna().sum()/len(df)*100:.1f}%)")
print(f"  ├─ 有价格数据: {df['price_level'].notna().sum()} ({df['price_level'].notna().sum()/len(df)*100:.1f}%)")
print(f"  └─ 平均评分: {df['rating'].mean():.2f}/5.0")
print(f"\n地理范围:")
print(f"  ├─ 纬度: {df['latitude'].min():.4f} ~ {df['latitude'].max():.4f}")
print(f"  └─ 经度: {df['longitude'].min():.4f} ~ {df['longitude'].max():.4f}")
print(f"\n输出文件:")
print(f"  └─ {output_filename}")
print("\n" + "=" * 60)
print("✅ 数据采集完成！可以用于Where to DINE项目")
print("=" * 60)